In [ ]:
import numpy as np # linear algebra
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import tqdm
import gc
import sys
import math
import tensorflow as tf
import keras
import keras.layers as L
from keras.models import Model,Sequential
from keras import regularizers
from sklearn.utils import class_weight
import re
from tensorflow.keras.utils import Sequence
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
train = pd.read_csv("../input/happy-whale-and-dolphin/train.csv")
train.head()

In [ ]:
sub = pd.read_csv('../input/happy-whale-and-dolphin/sample_submission.csv')
test_list = sub['image']

In [ ]:
train_base_addr = '../input/happy-whale-and-dolphin/train_images/'
test_base_addr = '../input/happy-whale-and-dolphin/test_images/'

In [ ]:
img = cv2.imread(train_base_addr + train['image'][0])
plt.imshow(img)
print("Mammal Id = " + train['individual_id'][0])

In [ ]:
img = cv2.imread(train_base_addr + train['image'][1])
plt.imshow(img)
print("Mammal Id = " + train['individual_id'][1])

In [ ]:
ID_list = train['individual_id'].to_list()
Unique_IDs = set(ID_list)
print("Number of Unique Dolphins/Whales Identified = " , len(Unique_IDs))
print("Total Number of Unique Dolphins/Whales Identified = " , len(ID_list))

In [ ]:
count = 10
sns.countplot(y="individual_id", data=train, palette="Greens_d",order=train.individual_id.value_counts().iloc[: count].index).set_title(f'Top {count} Whales/Dolphin sited')

In [ ]:
occurence_count = train.individual_id.value_counts()

In [ ]:
count = 20
top_occurences = len([i for i in occurence_count if i > count])
print(f"Number of Whales/Dolphin which occur more than {count} times = ", top_occurences)

In [ ]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder() 

In [ ]:
y = np.array(train['individual_id'])
label_encoded = label_encoder.fit_transform(y)
y = label_encoded

In [ ]:
y

In [ ]:
le_name_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
#print(le_name_mapping)

In [ ]:
reverse_le_name_mapping = dict(zip( label_encoder.transform(label_encoder.classes_), label_encoder.classes_))
#print(reverse_le_name_mapping)

In [ ]:
img_to_id_mapping = dict(zip(train["image"], train["individual_id"]))
#print(img_to_id_mapping)

In [ ]:
gc.collect()

In [ ]:
second_base_addr = '../input/happywhale-data/insta/'
second_test_addr = '../input/happywhale-test/insta/'

In [ ]:
class Dataset(Sequence):
    def __init__(self,idx,base_addr,directory,batch_size=16,shuffle=True):
        self.idx = idx
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.directory = directory
        self.base_addr = base_addr
        self.is_train = True
        self.y = []
        if self.directory == 'test':
            self.is_train = False
        
    def __len__(self):
        return math.ceil(len(self.idx)/self.batch_size)
    def getimg(self,x):
        idz = self.base_addr + x + '.npy'
        p = np.load(idz)
        #print(p.shape)
        #print(p)
        return p
    def getlabel(self,x):
        idz = img_to_id_mapping[x]
        le_label = le_name_mapping[idz]
        yz = le_label
       # print(y.shape)
        #print(y)
        return yz
                  
    def __getitem__(self,ids):
        batch_ids = self.idx[ids * self.batch_size:(ids + 1) * self.batch_size]    
        list_x1 = np.array([self.getimg(x) for x in batch_ids])/255
       # print(list_x1.shape)
        if self.directory != 'test':
            batch_y = np.array([self.getlabel(x) for x in batch_ids])
            #print(batch_y.shape)
            return [list_x1, batch_y], batch_y
        else:
            return list_x1

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train['image'], y, test_size=0.2, random_state=42)

In [ ]:
y_test

In [ ]:
train_dataset = Dataset(X_train,second_base_addr, 'train',32)
valid_dataset = Dataset(X_test,second_base_addr,'train',32)

In [ ]:
eval_dataset = Dataset(X_test,second_base_addr,'test',32)

In [ ]:
train_embed_dataset = Dataset(X_train,second_base_addr, 'test',32)

In [ ]:
test_dataset = Dataset(test_list,second_test_addr,'test',32)

In [ ]:
ef=tf.keras.applications.EfficientNetB6(input_shape=(128, 128, 3),weights='imagenet',include_top=False)

In [ ]:
ilr = 0.001

In [ ]:
class_weights = class_weight.compute_class_weight('balanced',np.unique(train['individual_id'].values),train['individual_id'].values)
class_weights = dict(enumerate(class_weights))
#class_weights

In [ ]:
from keras import backend as K
import math as m

In [ ]:
class ArcFace(keras.layers.Layer):
    def __init__(self, n_classes=15587, s=30.0, m=0.50, regularizer=None, **kwargs):
        super(ArcFace, self).__init__(**kwargs)
        self.n_classes = n_classes
        self.s = s
        self.m = m
        self.regularizer = regularizers.get(regularizer)
    def build(self, input_shape):
        super(ArcFace, self).build(input_shape[0])
        self.W = self.add_weight(name='W',
                                shape=(input_shape[0][-1], self.n_classes),
                                initializer='glorot_uniform',
                                trainable=True,
                                regularizer=self.regularizer)


    def call(self, inputs):
        x, y = inputs
        c = K.shape(x)[-1]
        
        x = tf.nn.l2_normalize(x, axis=1)
        
        W = tf.nn.l2_normalize(self.W, axis=0)
        
        logits = x @ W
        
        theta = tf.acos(K.clip(logits, -1.0 + K.epsilon(), 1.0 - K.epsilon()))
        target_logits = tf.cos(theta + self.m)

        logits = logits * (1 - y) + target_logits * y
        # feature re-scale
        logits *= self.s
        out = tf.nn.softmax(logits)
        return -1*out
    
    def compute_output_shape(self, input_shape):
        return (None, self.n_classes)


In [ ]:
def get_lr_callback(plot=False):
    lr_start   = 0.000001
    lr_max     = 0.000005 * 256 
    lr_min     = 0.000001
    lr_ramp_ep = 4
    lr_sus_ep  = 0
    lr_decay   = 0.9
   
    def lrfn(epoch):
        if epoch < lr_ramp_ep:
            lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start
            
        elif epoch < lr_ramp_ep + lr_sus_ep:
            lr = lr_max
            
        else:
            lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min
            
        return lr
        
    if plot:
        epochs = list(range(10))
        learning_rates = [lrfn(x) for x in epochs]
        plt.scatter(epochs,learning_rates)
        plt.show()

    lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=False)
    return lr_callback

get_lr_callback(plot=True)

In [ ]:
def crt_model():
    inp=L.Input(shape=(128, 128, 3))
    label =L.Input(shape=(15587,), )
    
    l = ef(inp)

    l = L.GlobalMaxPooling2D()(l)
    l = L.Dense(512, kernel_initializer='normal',activation='relu')(l)
    out = ArcFace(n_classes=15587)([l, label])
    model = Model(inputs= [inp, label],outputs=out)
    model.compile(optimizer=tf.keras.optimizers.Adam(ilr),loss=keras.losses.SparseCategoricalCrossentropy(), metrics=[tf.keras.metrics.SparseCategoricalAccuracy(),tf.keras.metrics.SparseTopKCategoricalAccuracy(k=5)])
    return model

In [ ]:
model = crt_model()
model.summary()

In [ ]:
gc.collect()

In [ ]:
history = model.fit(train_dataset, 
                    epochs = 10,
                    validation_data = valid_dataset,
                    class_weight = class_weights,
                   callbacks=[get_lr_callback()])

In [ ]:
gc.collect()

In [ ]:
embed_model = Model(inputs=model.input[0], outputs=model.layers[-3].output)

In [ ]:
embedded_features = embed_model.predict(eval_dataset, verbose=0)

In [ ]:
train_embed_features = embed_model.predict(train_embed_dataset, verbose=0)

In [ ]:
test_feature = embed_model.predict(test_dataset, verbose=0)

In [ ]:
train_embed_features.shape

In [ ]:
embedded_features.shape

In [ ]:
x_train = np.concatenate([train_embed_features,embedded_features])

In [ ]:
x_train.shape

In [ ]:
dd1 = {'embeddings' :x_train}

In [ ]:
dd2 = {'test_embeddings' :test_feature}

In [ ]:
np.save("embeddings", dd1)

In [ ]:
np.save("test_embeddings", dd2)